In [40]:
from data_loader import load_data
import numpy as np
import random

In [44]:
def calculate_cost(solution, num_depots, depot_capacities, setup_costs, customer_demands, cost_matrix):
    total_cost = 0
    depot_loads = np.zeros(num_depots)

    for customer, depot in enumerate(solution):
        depot_loads[depot] += customer_demands[customer]
        total_cost += cost_matrix[customer][depot]

    # Depo kurulum maliyetlerini ekle
    setup_costs = np.array(setup_costs)  # Eğer setup_costs bir liste ise, NumPy dizisine dönüştürüyoruz
    used_depots = depot_loads > 0
    total_cost += np.sum(setup_costs[used_depots])

    # Kapasite aşımı cezası
    capacity_excess = np.maximum(0, depot_loads - depot_capacities)
    penalty = np.sum(capacity_excess) * 1e4
    total_cost += penalty

    return total_cost

In [50]:
def greedy_initial_solution(num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix):
    """ Greedy algoritma ile başlangıç çözümü oluşturur ve maliyeti hesaplar. """
    solution = np.full(num_customers, -1, dtype=int)  # -1 atanmış olmayan müşterileri temsil eder
    remaining_capacities = depot_capacities.copy()

    for customer in range(num_customers):
        # Müşteri için en düşük maliyetli ve kapasiteye uygun depoyu bul
        feasible_depots = [
            (depot, cost_matrix[customer][depot])
            for depot in range(num_depots)
            if remaining_capacities[depot] >= customer_demands[customer]
        ]
        if feasible_depots:
            # En düşük maliyetli depoyu seç
            best_depot = min(feasible_depots, key=lambda x: x[1])[0]
            solution[customer] = best_depot
            remaining_capacities[best_depot] -= customer_demands[customer]
        else:
            # Eğer kapasitesi uygun depo yoksa rastgele bir atama yap
            solution[customer] = random.randint(0, num_depots - 1)

    # Çözümün toplam maliyetini hesapla
    cost = calculate_cost(solution, num_depots, depot_capacities, setup_costs, customer_demands, cost_matrix)

    return solution, cost

In [87]:
import numpy as np
import random

def tabu_search(solution,cost, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix,
                max_iterations=500, tabu_tenure=10, neighborhood_size=10):

    # Başlangıç çözümünü ve maliyetini al
    current_solution = solution.copy()
    current_cost = cost
    
    best_solution = current_solution.copy()
    best_cost = current_cost
    
    # Tabu listesi
    tabu_list = []

    for iteration in range(max_iterations):
        # Komşu çözümleri üret
        neighbors = generate_neighbors(current_solution, num_depots, neighborhood_size)
        best_neighbor = None
        best_neighbor_cost = float('inf')

        for neighbor in neighbors:
            neighbor_cost = calculate_cost(neighbor, num_depots, depot_capacities, setup_costs, customer_demands, cost_matrix)
            
            # Tabu listesinde olmayan ve maliyeti daha düşük olan çözümü seç
            if not any(np.array_equal(neighbor, tabu_item) for tabu_item in tabu_list) and (neighbor_cost < best_neighbor_cost):
                best_neighbor = neighbor
                best_neighbor_cost = neighbor_cost

        # Eğer daha iyi bir çözüm bulunmuşsa, güncelle
        if best_neighbor is not None:
            current_solution = best_neighbor
            current_cost = best_neighbor_cost
            
            # Tabu listesine ekle
            tabu_list.append(best_neighbor)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)  # Tabu listesinin boyutunu sınırla
            
            # Genel en iyi çözümü güncelle
            if best_neighbor_cost < best_cost:
                best_solution = best_neighbor
                best_cost = best_neighbor_cost

    return best_solution, best_cost

def generate_neighbors(solution, num_depots, neighborhood_size):
    """
    Komşu çözümleri üretir.
    """
    neighbors = []
    for _ in range(neighborhood_size):
        new_solution = solution.copy()
        customer_to_change = random.randint(0, len(solution) - 1)
        new_solution[customer_to_change] = random.randint(0, num_depots - 1)
        neighbors.append(new_solution)
    return neighbors

In [65]:
def simulated_annealing(current_solution,current_cost,num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix,
                        initial_temperature=1000, cooling_rate=0.95, max_iterations=1000):
    
    best_solution = current_solution.copy()
    best_cost = current_cost.copy()

    temperature = initial_temperature

    for iteration in range(max_iterations):
        # Rastgele bir komşu çözüm oluştur
        new_solution = current_solution.copy()
        customer_to_change = random.randint(0, num_customers - 1)
        new_solution[customer_to_change] = random.randint(0, num_depots - 1)

        new_cost = calculate_cost(new_solution, num_depots, depot_capacities, setup_costs, customer_demands, cost_matrix)

        # Kabul kriterini kontrol et
        if new_cost < current_cost or random.random() < np.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost

            # En iyi çözümü güncelle
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        # Sıcaklığı azalt
        temperature *= cooling_rate

     
    return best_solution, best_cost

In [47]:
import numpy as np
import random

class GeneticAlgorithm:
    def __init__(self, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix,
                 population_size=100, generations=500, mutation_rate=0.1, crossover_rate=0.8):
        self.num_depots = num_depots
        self.num_customers = num_customers
        self.depot_capacities = np.array(depot_capacities)
        self.setup_costs = np.array(setup_costs)
        self.customer_demands = np.array(customer_demands)
        self.cost_matrix = np.array(cost_matrix)
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

    def initialize_population(self):
        # Each individual is a list of integers, where index is the customer and value is the depot assignment
        return [np.random.randint(0, self.num_depots, size=self.num_customers) for _ in range(self.population_size)]

    def fitness(self, individual):
        # Calculate the total cost for a given depot-customer assignment
        total_cost = 0
        depot_loads = np.zeros(self.num_depots)

        for customer, depot in enumerate(individual):
            depot_loads[depot] += self.customer_demands[customer]
            total_cost += self.cost_matrix[customer][depot]

        # Add depot setup costs for depots that are used
        used_depots = depot_loads > 0
        total_cost += np.sum(self.setup_costs[used_depots])

        # Penalize solutions that exceed depot capacities
        capacity_excess = np.maximum(0, depot_loads - self.depot_capacities)
        penalty = np.sum(capacity_excess) * 1e4  # Ceza katsayısını 1e4'e düşürdük
        total_cost += penalty

        return total_cost

    def selection(self, population, fitness_scores):
        # Turnuva Seçimi
        tournament_size = 5
        selected = random.sample(list(zip(population, fitness_scores)), tournament_size)
        selected = sorted(selected, key=lambda x: x[1])  # Fitness'e göre sırala
        return selected[0][0], selected[1][0]
    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            # Single-point crossover
            point = random.randint(1, self.num_customers - 1)
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        return parent1.copy(), parent2.copy()

    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            # Swap mutasyonu: İki müşterinin depot atamasını değiştir
            customer1, customer2 = random.sample(range(self.num_customers), 2)
            individual[customer1], individual[customer2] = individual[customer2], individual[customer1]
        return individual

    def evolve(self):
        population = self.initialize_population()
        best_solution = None
        best_cost = float('inf')

        for generation in range(self.generations):
            fitness_scores = [self.fitness(individual) for individual in population]
            new_population = []

            for _ in range(self.population_size // 2):
                parent1, parent2 = self.selection(population, fitness_scores)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                new_population.extend([child1, child2])

            population = new_population

            # Track the best solution
            min_cost = min(fitness_scores)
            if min_cost < best_cost:
                best_cost = min_cost
                best_solution = population[np.argmin(fitness_scores)]

        early_stop_generations = 50  # Son 50 nesilde iyileşme olmazsa durdur
        no_improvement_count = 0
        for generation in range(self.generations):
            fitness_scores = [self.fitness(individual) for individual in population]
            min_cost = min(fitness_scores)
            
            if min_cost < best_cost:
                best_cost = min_cost
                best_solution = population[np.argmin(fitness_scores)]
                no_improvement_count = 0
            else:
                no_improvement_count += 1
            
            if no_improvement_count >= early_stop_generations:
                print(f"Erken durdurma: İyileşme görülmedi. {generation + 1}. nesilde durduruldu.")
                break
        return best_solution, best_cost

In [91]:
def run_genetic_with_sa(sa_solution, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix):
    # Genetik algoritmayı başlat
    ga = GeneticAlgorithm(
        num_depots=num_depots,
        num_customers=num_customers,
        depot_capacities=depot_capacities,
        setup_costs=setup_costs,
        customer_demands=customer_demands,
        cost_matrix=cost_matrix,
        population_size=1500,  # Popülasyon boyutu
        generations=1500,  # Nesil sayısı
        mutation_rate=0.5,  # Mutasyon oranı
        crossover_rate=1.0  # Çaprazlama oranı
    )

    # SA çözümünü popülasyona ekle
    population = ga.initialize_population()
    population[0] = sa_solution  # İlk birey SA çözümü
    ga.population = population

    # Genetik algoritmayı çalıştır
    best_solution, best_cost = ga.evolve()
    return best_solution, best_cost

In [ ]:
def improved_local_search(solution, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix, 
                          max_iterations=500, tabu_tenure=10):
    best_solution = solution.copy()
    best_cost = calculate_cost(best_solution, num_depots, depot_capacities, setup_costs, customer_demands, cost_matrix)

    # Tabu listesi
    tabu_list = []

    for iteration in range(max_iterations):
        # En iyi komşuyu bul
        neighbors = generate_neighbors_local_search(best_solution, num_depots)
        best_neighbor = None
        best_neighbor_cost = float('inf')

        for neighbor in neighbors:
            if not any(np.array_equal(neighbor, tabu_item) for tabu_item in tabu_list):
                neighbor_cost = calculate_cost(neighbor, num_depots, depot_capacities, setup_costs, customer_demands, cost_matrix)
                if neighbor_cost < best_neighbor_cost:
                    best_neighbor = neighbor
                    best_neighbor_cost = neighbor_cost

        # Eğer en iyi komşu mevcut çözümden daha iyiyse, güncelle
        if best_neighbor_cost < best_cost:
            best_solution = best_neighbor
            best_cost = best_neighbor_cost

        # Tabu listesine ekle
        tabu_list.append(best_neighbor)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Tabu listesi boyutunu kontrol et

    return best_solution, best_cost

def generate_neighbors_local_search(solution, num_depots):
    """Komşu çözümleri üretir."""
    neighbors = []
    for _ in range(10):  # 10 komşu çözüm üret
        new_solution = solution.copy()
        customer_to_change = random.randint(0, len(solution) - 1)
        new_solution[customer_to_change] = random.randint(0, num_depots - 1)
        neighbors.append(new_solution)
    return neighbors


In [89]:
# Load the problem data
file_path = "Dataset/wl_25"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)
assert len(setup_costs) == num_depots, "setup_costs boyutu depo sayısıyla uyumlu değil!"

solution, cost = greedy_initial_solution(num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix)
print("Greedy Initial Solution Cost:", cost)
tabu_solution,tabu_cost = tabu_search(solution,cost,num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix)
print("Tabu Search Solution Cost:", tabu_cost)

ga_solution, ga_cost = run_genetic_with_sa(tabu_solution, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix)
print("GA Solution Cost:", ga_cost)


Greedy Initial Solution Cost: 832291.15
Tabu Search Solution Cost: 827594.4625000001
Erken durdurma: İyileşme görülmedi. 50. nesilde durduruldu.
GA Solution Cost: 807025.95


In [92]:
# Load the problem data
file_path = "Dataset/wl_500"  # Replace with your actual data file path
num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix = load_data(file_path)
assert len(setup_costs) == num_depots, "setup_costs boyutu depo sayısıyla uyumlu değil!"

solution, cost = greedy_initial_solution(num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix)
print("Greedy Initial Solution Cost:", cost)
tabu_solution,tabu_cost = tabu_search(solution,cost,num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix)
print("Tabu Search Solution Cost:", tabu_cost)

ga_solution, ga_cost = run_genetic_with_sa(tabu_solution, num_depots, num_customers, depot_capacities, setup_costs, customer_demands, cost_matrix)
print("GA Solution Cost:", ga_cost)


Greedy Initial Solution Cost: 99795.03574
Tabu Search Solution Cost: 54154.08539
Erken durdurma: İyileşme görülmedi. 50. nesilde durduruldu.
GA Solution Cost: 20005.62322
